In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
import time

In [2]:
def clean_movie(movie):
    
    movie = dict(movie)
    alt_titles = {}
    # Combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # Merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [18]:
def function():
    kaggle_metadata = pd.read_csv(r'C:\Users\selin\OneDrive\Desktop\ColinBootcamp\movies_metadata.csv', low_memory = False)
    ratings = pd.read_csv(r"C:\Users\selin\OneDrive\Desktop\ColinBootcamp\ratings.csv")
    with open(r"C:\Users\selin\OneDrive\Desktop\ColinBootcamp\wikipedia-movies.json", mode = 'r') as file:
        movies = json.load(file)
        
    wiki_movies = [movie for movie in movies if ('Director' in movie or 'Directed by' in movie) and 'imdb_link' in movie and 'No. of episodes' not in movie]
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    
    Wmovies_df = pd.DataFrame(clean_movies)
    
    try:
        Wmovies_df['imdb_id'] = Wmovies_df['imdb_link'].str.extract(r'(tt\d{7})')
        Wmovies_df.drop_duplicates(subset = 'imbd_id', inplace = True)
    except Exception as e:
        print(e)
        
    columns_to_keep = [column for column in Wmovies_df.columns if Wmovies_df[column].isnull().sum() < len(Wmovies_df) * 0.9]
    Wmovies_df = Wmovies_df[columns_to_keep]
    
    box_office = Wmovies_df['Box office'].dropna()
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    form_one = 1
    form_two = 2
    
    def parse_dollars(s):
        
    # If s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # If input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # Remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # Convert to float and multiply by a million
            value = float(s) * 10**6

        # Return value
            return value

    # If input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # Remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # Convert to float and multiply by a billion
            value = float(s) * 10**9

        # Return value
            return value

    # If input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # Remove dollar sign and commas
            s = re.sub('\$|,','', s)

        # Convert to float
            value = float(s)

        # Return value
            return value
    
    # Otherwise, return NaN
        else:
            return np.nan

    budget = Wmovies_df['Budget'].dropna() 
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x) 
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True) 
    Wmovies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    release_date = Wmovies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x) 
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}' 
    date_form_two = r'\d{4}.[01]\d.[123]\d' 
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}' 
    date_form_four = r'\d{4}' 
    Wmovies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True) 
    
    
    running_time = Wmovies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x) 
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m') 
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0) 
    Wmovies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1) 
    Wmovies_df.drop('Running time', axis=1, inplace=True) 
    
    
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns') 
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True' 
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int) 
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise') 
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise') 
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date']) 
    
    
    movies_df = pd.merge(Wmovies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle']) 
    
    
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True) 
   

    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column] , axis=1) 
        df.drop(columns=wiki_column, inplace=True) 
        
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time') 
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki') 
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    

    movies_df = movies_df[['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link', 'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count', 'genres','original_language','overview','spoken_languages','Country', 'production_companies','production_countries','Distributor', 'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on' ]] 
    
    
    movies_df.rename({'id':'kaggle_id', 'title_kaggle':'title', 'url':'wikipedia_url', 'budget_kaggle':'budget', 'release_date_kaggle':'release_date', 'Country':'country', 'Distributor':'distributor', 'Producer(s)':'producers', 'Director':'director', 'Starring':'starring', 'Cinematography':'cinematography', 'Editor(s)':'editors', 'Writer(s)':'writers', 'Composer(s)':'composers', 'Based on':'based_on' }, axis='columns', inplace=True) 
    
    
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count().rename({'userId':'count'}, axis=1).pivot(index='movieId',columns='rating', values='count') 
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns] 
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left') 
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0) 
    
    return wiki_movies_df, movies_with_ratings_df, movies_df  

In [19]:
file_dir = r"C:\Users\selin\OneDrive\Desktop\ColinBootcamp"
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file = f'{file_dir}/ratings.csv'

In [20]:
wiki_file,kaggle_file,ratings_file = function()

Index(['imbd_id'], dtype='object')


ValueError: Columns must be same length as key

In [12]:
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [13]:
wiki_movies_df.head()

AttributeError: 'str' object has no attribute 'head'

In [14]:
movies_with_ratings_df.head()

AttributeError: 'str' object has no attribute 'head'